Use the Newer Version of Parmest

Tasks:
Run through the example here:
https://dowlinglab.github.io/pyomo-doe/notebooks/parmest.html

onvert old code to this design model:
https://github.com/Pyomo/pyomo/blob/main/pyomo/contrib/parmest/examples/reactor_design/reactor_design.py

In [1]:
# Installing libraries
import sys
!pip install pyomo
import pyomo
import scipy

if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/optimization/main/notebooks/helper.py"
    import helper
    helper.easy_install()
else:
    sys.path.insert(0, '../')
    import helper
helper.set_plotting_style()

ModuleNotFoundError: No module named 'helper'

Non Linear Regression with Parmest

In [2]:
import sys

# If running on Google Colab, install Pyomo and Ipopt via IDAES
on_colab = "google.colab" in sys.modules
if on_colab:
    !wget "https://raw.githubusercontent.com/dowlinglab/pyomo-doe/main/notebooks/tclab_pyomo.py"

# import TCLab model, simulation, and data analysis functions
from tclab_pyomo import (
    TC_Lab_data,
    TC_Lab_experiment,
    extract_results,
    extract_plot_results,
)

# set default number of states in the TCLab model
number_tclab_states = 2

ModuleNotFoundError: No module named 'tclab_pyomo'

In [ ]:
import pandas as pd

if on_colab:
    file = "https://raw.githubusercontent.com/dowlinglab/pyomo-doe/main/data/tclab_sine_test_5min_period.csv"
else:
    file = '../data/tclab_sine_test_5min_period.csv'
df = pd.read_csv(file)
df.head()

In [ ]:
ax = df.plot(x='Time', y=['T1', 'T2'], xlabel='Time (s)', ylabel='Temperature (°C)')
ax = df.plot(x='Time', y=['Q1', 'Q2'], xlabel='Time (s)', ylabel='Heater Power (%)')

In [ ]:
tc_data = TC_Lab_data(
    name="Sine Wave Test for Heater 1",
    time=df['Time'].values,
    T1=df['T1'].values,
    u1=df['Q1'].values,
    P1=200,
    TS1_data=None,
    T2=df['T2'].values,
    u2=df['Q2'].values,
    P2=200,
    TS2_data=None,
    Tamb=df['T1'].values[0],
)
tc_data.to_data_frame().head()

In [ ]:
import pyomo.contrib.parmest.parmest as parmest

# First, we define an Experiment object within parmest
TC_Lab_sine_exp = TC_Lab_experiment(data=tc_data, number_of_states=number_tclab_states)

# Since everything has been labeled properly in the Experiment object, we simply invoke
# parmest's Estimator function to estimate the parameters.
pest = parmest.Estimator([TC_Lab_sine_exp, ], obj_function='SSE', tee=True)

obj, theta = pest.theta_est()

In [ ]:
parmest_regression_results = extract_plot_results(
    tc_data, pest.ef_instance
)

In [ ]:
theta_values = theta.to_dict()
print("Estimated parameters:\n", theta_values)

Exercise

In [ ]:
import sys

# If running on Google Colab, install Pyomo and Ipopt via IDAES
on_colab = "google.colab" in sys.modules
if on_colab:
    !wget "https://raw.githubusercontent.com/dowlinglab/pyomo-doe/main/notebooks/tclab_pyomo.py"
else:
    import os

    if "exercise_solutions" in os.getcwd():
        # Add the "notebooks" folder to the path
        # This is needed for running the solutions from a separate folder
        # You only need this if you run locally
        sys.path.append('../notebooks')

# import TCLab model, simulation, and data analysis functions
from tclab_pyomo import (
    TC_Lab_data,
    TC_Lab_experiment,
    extract_results,
    extract_plot_results,
)

# set default number of states in the TCLab model
number_tclab_states = 2

In [ ]:
import pandas as pd

if on_colab:
    file = "https://raw.githubusercontent.com/dowlinglab/pyomo-doe/main/data/tclab_step_test.csv"
else:
    file = '../data/tclab_step_test.csv'
df = pd.read_csv(file)
df.head()

In [ ]:
ax = df.plot(x='Time', y=['T1', 'T2'], xlabel='Time (s)', ylabel='Temperature (°C)')
ax = df.plot(x='Time', y=['Q1', 'Q2'], xlabel='Time (s)', ylabel='Heater Power (%)')

In [ ]:
tc_data = TC_Lab_data(
    name="Step Test for Heater 1",
    time=df['Time'].values,
    T1=df['T1'].values,
    u1=df['Q1'].values,
    P1=200,
    TS1_data=None,
    T2=df['T2'].values,
    u2=df['Q2'].values,
    P2=200,
    TS2_data=None,
    Tamb=df['T1'].values[0],
)

tc_data.to_data_frame().head()

In [ ]:
import pyomo.contrib.parmest.parmest as parmest

# First, we define an Experiment object within parmest
# Add your solution here
TC_Lab_sine_exp = TC_Lab_experiment(data=tc_data, number_of_states=number_tclab_states)

# Since everything has been labeled properly in the Experiment object, we simply invoke
# parmest's Estimator function to estimate the parameters.
# Add your solution here
pest = parmest.Estimator([TC_Lab_sine_exp, ], obj_function='SSE', tee=True)
obj, theta = pest.theta_est()

parmest_regression_results = extract_plot_results(
    tc_data, pest.ef_instance
)

Experiment Class with PCB

In [ ]:
from pyomo.common.dependencies import pandas as pd
import pyomo.environ as pyo
import pyomo.contrib.parmest.parmest as parmest
from pyomo.contrib.parmest.experiment import Experiment
import numpy as np


# Data Preparation
def prepare_data():
    data = {
        'Age (years)': [1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 11, 12, 12, 12],
        'PCB Concentration (ppm)': [
            0.6, 1.6, 0.5, 1.2, 2.0, 1.3, 2.5, 2.2, 2.4, 1.2,
            3.5, 4.1, 5.1, 5.7, 3.4, 9.7, 8.6, 4.0, 5.5, 10.5,
            17.5, 13.4, 4.5, 30.4, 12.4, 13.4, 26.2, 7.4
        ]
    }
    df = pd.DataFrame(data)
    df['ln_PCB_Concentration'] = np.log(df['PCB Concentration (ppm)'])
    return df


# Define the Pyomo model
def pcb_concentration_model(data):
    model = pyo.ConcreteModel()

    # Define variables
    model.m = pyo.Var(initialize=0)  # Slope
    model.b = pyo.Var(initialize=0)  # Intercept

    # Define parameters
    model.age = pyo.Param(within=pyo.NonNegativeReals, mutable=True, initialize=data['Age (years)'])
    model.ln_pcb = pyo.Param(within=pyo.Reals, mutable=True, initialize=data['ln_PCB_Concentration'])

    # Define objective function
    def objective_rule(m):
        return (m.ln_pcb - (m.m * m.age + m.b)) ** 2
    model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

    return model


# Experiment class for Parmest
class PCBConcentrationExperiment(Experiment):

    def __init__(self, data, experiment_number):
        self.data = data
        self.experiment_number = experiment_number
        self.data_i = data.iloc[experiment_number]
        self.model = None

    def create_model(self):
        self.model = m = pcb_concentration_model()
        return m

    def finalize_model(self):
        m = self.model
        m.age = self.data_i['Age (years)']
        m.ln_pcb = self.data_i['ln_PCB_Concentration']
        return m

    def label_model(self):
        m = self.model
        m.experiment_outputs = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.unknown_parameters = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.unknown_parameters.update({m.m: 'm', m.b: 'b'})
        return m

    def get_labeled_model(self):
        m = self.create_model()
        m = self.finalize_model()
        m = self.label_model()
        return m


def estimate_parameters():
    # Prepare data
    df = prepare_data()

    # Create experiment list for Parmest
    exp_list = [
        {'Age (years)': row['Age (years)'], 'ln_PCB_Concentration': row['ln_PCB_Concentration']}
        for _, row in df.iterrows()
    ]

    # Define SSE objective function for Parmest
    def SSE(model, data):
        return (data['ln_PCB_Concentration'] - (model.m * data['Age (years)'] + model.b)) ** 2

    # Create estimator instance
    pest = parmest.Estimator(pcb_concentration_model, exp_list, ['m', 'b'], obj_function=SSE)

    estimate_results = pest.theta_est()
    print(estimate_results)


def main():
    estimate_parameters()


if __name__ == "__main__":
    main()

Experiment Class for Puromycin

In [ ]:
from pyomo.common.dependencies import pandas as pd
import pyomo.environ as pyo
import pyomo.contrib.parmest.parmest as parmest
from pyomo.contrib.parmest.experiment import Experiment
import numpy as np


# Data Preparation
def prepare_data():
    # Provided data
    substrate_concentration = np.array([0.02, 0.06, 0.11, 0.22, 0.56, 1.10])
    treated_velocity = np.array([[76, 47], [97, 107], [123, 139], [159, 152], [191, 201], [207, 200]])
    untreated_velocity = np.array([[67, 51], [84, 86], [98, 115], [131, 124], [144, 158], [160, np.nan]])

    # Calculate the average velocities
    treated_avg_velocity = np.nanmean(treated_velocity, axis=1)
    untreated_avg_velocity = np.nanmean(untreated_velocity, axis=1)

    return substrate_concentration, treated_avg_velocity, untreated_avg_velocity


# Define the Pyomo Michaelis-Menten Model
def michaelis_menten_model(data=None):  # Add data argument with default value None
    model = pyo.ConcreteModel()

    # Define variables
    model.Vmax = pyo.Var(within=pyo.NonNegativeReals, initialize=200)  # Maximum reaction velocity
    model.Km = pyo.Var(within=pyo.NonNegativeReals, initialize=0.1)  # Michaelis constant

    if data:
        model.substrate_concentration = pyo.Param(initialize=data.get('substrate_concentration', 0))
        model.velocity = pyo.Param(initialize=data.get('velocity', 0))

    return model


# Experiment class
class MichaelisMentenExperiment(Experiment):

    def __init__(self, data, experiment_number):
        self.data = data
        self.experiment_number = experiment_number
        self.data_i = data[experiment_number]
        self.model = None

    def create_model(self):
        self.model = m = michaelis_menten_model()
        return m

    def finalize_model(self):
        m = self.model
        m.substrate_concentration = self.data_i.get('substrate_concentration', 0)
        m.velocity = self.data_i.get('velocity', 0)
        return m

    def label_model(self):
        m = self.model
        m.experiment_outputs = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.unknown_parameters = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.unknown_parameters.update({m.Vmax: 'Vmax', m.Km: 'Km'})
        return m

# Prepare the experiment data for Parmest
def prepare_experiment_data(substrate_concentration, velocity_data):
    return [
        {'substrate_concentration': substrate_concentration[i], 'velocity': velocity_data[i]}
        for i in range(len(substrate_concentration))
    ]


# Define SSE function for Parmest
def SSE(model, data):
    # Calculate the predicted velocity using the Michaelis-Menten equation
    predicted_velocity = (model.Vmax * data['substrate_concentration']) / \
                         (model.Km + data['substrate_concentration'])
    # Calculate the squared error for the current data point
    error = (data['velocity'] - predicted_velocity)**2
    return error


# Parameter Estimation Function
def estimate_parameters():
    # Prepare data
    substrate_concentration, treated_avg_velocity, untreated_avg_velocity = prepare_data()

    # Create experiment lists for treated and untreated data
    treated_exp_list = prepare_experiment_data(substrate_concentration, treated_avg_velocity)
    untreated_exp_list = prepare_experiment_data(substrate_concentration, untreated_avg_velocity)

    # Create Parmest estimators
    pest_treated = parmest.Estimator(
        michaelis_menten_model, treated_exp_list, ['Vmax', 'Km'], obj_function=SSE
    )
    pest_untreated = parmest.Estimator(
        michaelis_menten_model, untreated_exp_list, ['Vmax', 'Km'], obj_function=SSE
    )

    # Estimate parameters for treated data
    obj_treated, theta_treated = pest_treated.theta_est()
    print(f'Treated Model - Vmax: {theta_treated["Vmax"]:.2f}, Km: {theta_treated["Km"]:.4f}')

    # Estimate parameters for untreated data
    obj_untreated, theta_untreated = pest_untreated.theta_est()
    print(f'Untreated Model - Vmax: {theta_untreated["Vmax"]:.2f}, Km: {theta_untreated["Km"]:.4f}')


def main():
    estimate_parameters()


if __name__ == "__main__":
    main()